In [1]:
import kaggle
!kaggle datasets download shivamb/netflix-shows -f netflix_titles.csv --force

Dataset URL: https://www.kaggle.com/datasets/shivamb/netflix-shows
License(s): CC0-1.0




  0%|          | 0.00/1.34M [00:00<?, ?B/s]
100%|##########| 1.34M/1.34M [00:00<00:00, 431MB/s]


In [7]:
%pwd

'C:\\Users\\Umar\\Desktop\\Project_2025\\ELT_Netflix'

In [5]:
import pandas as pd
df = pd.read_csv('netflix_titles.csv')

In [19]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [15]:
import sqlalchemy as sal
import pyodbc 
import urllib 
# --- Your pyodbc Connection Details ---
server_name = r'LAPTOP-6CUNETER\SQLEXPRESS'  # Raw string for server name
database_name = 'master'
driver = '{ODBC Driver 17 for SQL Server}'   # Use the braces for pyodbc

# Assuming Windows Authentication as it's common and was in previous examples
# If using SQL Server Authentication, you'd add UID=YourUsername;PWD=YourPassword;
pyodbc_conn_str = f'DRIVER={driver};SERVER={server_name};DATABASE={database_name};Trusted_Connection=yes;'
print(f"Original pyodbc connection string: {pyodbc_conn_str}")

# --- Create SQLAlchemy Engine using the pyodbc connection string ---
# URL encode the pyodbc connection string
params = urllib.parse.quote_plus(pyodbc_conn_str)
sqlalchemy_engine_str = f"mssql+pyodbc:///?odbc_connect={params}"
print(f"SQLAlchemy engine string: {sqlalchemy_engine_str}")
engine = sal.create_engine(sqlalchemy_engine_str)

Original pyodbc connection string: DRIVER={ODBC Driver 17 for SQL Server};SERVER=LAPTOP-6CUNETER\SQLEXPRESS;DATABASE=master;Trusted_Connection=yes;
SQLAlchemy engine string: mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BODBC+Driver+17+for+SQL+Server%7D%3BSERVER%3DLAPTOP-6CUNETER%5CSQLEXPRESS%3BDATABASE%3Dmaster%3BTrusted_Connection%3Dyes%3B


In [23]:
# load the data into sql server
df.to_sql('netflix_raw', engine, index=False, if_exists = 'append') # Use 'engine' here

107

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [11]:
df.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [9]:
# Code to check the maximum size required by each col
max_lengths = {}
for col in df.columns:
    max_lengths[col] = df[col].astype(str).apply(len).max()

for col, length in max_lengths.items():
    print(f"  Column '{col}': {length}")


  Column 'show_id': 5
  Column 'type': 7
  Column 'title': 104
  Column 'director': 208
  Column 'cast': 771
  Column 'country': 123
  Column 'date_added': 19
  Column 'release_year': 4
  Column 'rating': 8
  Column 'duration': 10
  Column 'listed_in': 79
  Column 'description': 248


In [17]:
# load the data into sql server
df.to_sql('netflix_raw', engine, index=False, if_exists = 'append') # Use 'engine' here

107

In [19]:
# load the data into sql server
df.to_sql('netflix_raw', engine, index=False, if_exists = 'append') # Use 'engine' here

107